In [1]:
# CNN for time series data

# GPU 
GPU = False
if GPU:
    import cupy as np
    np.cuda.set_allocator(np.cuda.MemoryPool().malloc)
else:
    import numpy as np

In [4]:
# Fully Connected Layer
class FullyConnected:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        W, b = self.params

        y = np.dot(x, W) + b        # y = X*W + b
        self.x = x

        return y

    def backward(self, dy):
        W, b = self.params
        x = self.x

        db = np.sum(dy, axis=0)
        dW = np.dot(x.T, dy)
        dx = np.dot(dy, W.T)

        self.grads[0][...] = dW
        self.grads[1][...] = db

        return dx

In [5]:
# CNN Layer (Conv, Pooling)

def im2col(data, filter_h, filter_w, stride = 1, padding = 0):
    # flatten data to 2D array
    N, C, H, W = data.shape
    
    out_h = (H + 2*padding - filter_h)//stride + 1
    out_w = (W + 2*padding - filter_w)//stride + 1

    # padding for H, W
    img = np.pad(data, 
                 [(0, 0), (0, 0), (padding, padding), (padding, padding)], 
                 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)

    return col

def col2im(col, shape, filter_h, filter_w, stride=1, padding=0):
    # 2D for img data
    # shape = original data shape
    N, C, H, W = shape
    out_h = (H + 2*padding - filter_h)//stride + 1
    out_w = (W + 2*padding - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zero((N, C, H+2*padding + stride -1, W+2*padding+stride-1))
    

class Convolution:
    def __init__(self, W, b, stride=1, padding=0):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.stride = stride
        self.padding = padding
        self.cache = None

    def forward(self, x):
        W, b = self.params
        FN, FC, FH, FW = W.shape
        N, C, H, W = x.shape        # Samples, Channel, Time steps(24), Features

        out_h = (H + 2*self.padding - FH)//self.stride + 1
        out_w = (W + 2*self.padding - FW)//self.stride + 1

        col = im2col(x, FH, FW, self.stride, self.padding)
        col_W = self.W.reshape(FN, -1).T

        y = np.dot(col, col_W) + b
        y = y.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.cache = (x, col, col_W)

        return y

    def backward(self, dy):
        W, b = self.params
        x, col, col_W = self.cache
        FN, C, FH, FW = W.shape
        
        dy = dy.transpose(0, 2, 3, 1).reshape(-1, FN)

        self.db = np.sum(dy, axis=0)
        self.dW = np.dot(col.T, dy)
        self.dx = np.dot(dy, col_W.T)
        dx = col2im(dx, x.shape, FH, FW, self.stride, self.padding)

        return dx


In [16]:
x= np.random.randn(10, 1, 12, 8)
px = np.pad(x, [(0, 0), (0, 0), (1, 1), (1, 1)], 'constant')
print(px)

[[[[ 0.          0.          0.         ...  0.          0.
     0.        ]
   [ 0.          0.52958955 -0.25605225 ...  0.06196239  1.20846192
     0.        ]
   [ 0.         -0.23320859 -0.82458087 ... -0.27868601  1.15410825
     0.        ]
   ...
   [ 0.         -0.35425941  1.50275483 ... -0.5046609   1.24949429
     0.        ]
   [ 0.         -0.89840033  0.48048885 ... -0.75633106  0.05884447
     0.        ]
   [ 0.          0.          0.         ...  0.          0.
     0.        ]]]


 [[[ 0.          0.          0.         ...  0.          0.
     0.        ]
   [ 0.          1.81405763  1.47187117 ... -2.07509542 -1.30188557
     0.        ]
   [ 0.         -0.80160177 -0.41085056 ...  1.34805632 -0.46092427
     0.        ]
   ...
   [ 0.          0.03108273  1.14974657 ... -0.75552772 -0.53635475
     0.        ]
   [ 0.         -0.59317936 -0.20582094 ...  0.15673816 -0.11925074
     0.        ]
   [ 0.          0.          0.         ...  0.          0.
     0.    